In [2]:
# Importamos las librerías

import tensorflow as tf
from PIL import Image
import os
import matplotlib.pyplot as plt
import numpy as np
import tifffile
import sys
import torch.nn as nn
import torch

In [ ]:
!python -m pip install tensorflow
!python -m pip install keras
!python -m pip install numpy
!python -m pip install Pillow
!python -m pip install os
!python -m pip install scipy
!python -m pip install scikit-learn
!python -m pip install matplotlib.pyplot
!python -m pip install torch

In [24]:
ruta_no_marcadas=r'C:\Users\unidad-epa\Desktop\Iker\Scripts\Reconstrucción defectos\Defecto_1'
ruta_marcados=r'C:\Users\unidad-epa\Desktop\Iker\Scripts\Reconstrucción defectos\Defecto_1_marcado'
imagenes_tif_marcado=[]
imagenes_tif=[]

def cargar_imagenes(ruta):
  contenedor=[]
  for archivo in os.listdir(ruta):
    ruta_completa=os.path.join(ruta, archivo)
    if os.path.isfile(ruta_completa):
      imagen=Image.open(ruta_completa)
      contenedor.append(imagen)
    return contenedor
  

archivos_tif = np.array(cargar_imagenes(ruta_no_marcadas))/255
archivos_tif.max()
#imagenes_tif_marcado = np.array(cargar_imagenes(ruta_marcados, imagenes_tif_marcado))/255

np.float64(1.0)

In [9]:
imagenes_tif_marcado

array([], dtype=float64)

In [ ]:
from PIL import Image
from PIL.TiffTags import TAGS

# Cargar imagen
imagen = Image.open('/content/Data/Defectos/ProbetaAE_008.tif')

# Obtener metadatos
metadatos = imagen.tag_v2
for tag_id, valor in metadatos.items():
    nombre_tag = TAGS.get(tag_id, tag_id)
    print(f"{nombre_tag}: {valor}")

ImageWidth: 991
ImageLength: 1022
BitsPerSample: (8, 8, 8)
Compression: 32773
PhotometricInterpretation: 2
StripOffsets: (8, 440, 872, 1304, 1736, 2168, 2600, 3032, 3464, 3896, 4328, 4760, 5192, 5624, 6056, 6488, 6920, 7352, 7784, 8216, 8648, 9080, 9512, 9944, 10376, 10812, 12560, 15906, 20228, 25164, 30548, 36422, 42794, 49444, 56476, 63610, 71064, 78894, 86930, 95314, 103940, 112738, 121712, 130808, 140146, 149574, 159140, 168786, 178580, 188520, 198478, 208480, 218564, 228708, 238878, 249100, 259302, 269550, 279820, 290050, 300278, 310412, 320508, 330528, 340546, 350446, 360256, 369970, 379598, 389108, 398480, 407782, 416848, 425770, 434492, 443038, 451366, 459490, 467320, 474884, 482190, 489108, 495690, 501946, 507636, 512860, 517538, 521472, 524368, 525548, 525980, 526412, 526844, 527276, 527708, 528140, 528572, 529004, 529436, 529868, 530300, 530732, 531164, 531596, 532028, 532460, 532892, 533324, 533756, 534188, 534620, 535124, 535625, 536411)
SampleFormat: (1, 1, 1)
SamplesPerP

Vamos a probar a hacer la reconstrucción con una red neuronal separada

In [ ]:
def crop_to_match(encoder_tensor, decoder_tensor):
    d, h, w = decoder_tensor.shape[2:]
    return encoder_tensor[:, :, :d, :h, :w]

In [ ]:
# Cargamos las imágenes clasificadas como defectuosas

defectos = np.stack(imagenes_tif, axis=0)

# Definimos el modelo

class UNet3D(nn.Module):
    def __init__(self, in_channels=1, out_channels=1): # Constructor de la clase
      super().__init__()
      def CBR(in_c, out_c):
        return nn.Sequential(
            nn.Conv3d(in_c, out_c, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(out_c, out_c, 3, padding=1),
            nn.ReLU(inplace=True)
        )
      self.enc1 = CBR(in_channels, 32)
      self.pool1 = nn.MaxPool3d(2)
      self.enc2 = CBR(32, 64)
      self.pool2 = nn.MaxPool3d(2)

      self.bottleneck = CBR(64, 128)

      # Añadimos output_padding=(1, 0, 0) para corregir un posible desfase de 1 voxel en la dimensión de profundidad
      # entre el upsampling y el tensor del encoder correspondiente.
      self.up2 = nn.ConvTranspose3d(128, 64, 2, stride=2, output_padding=(1, 0, 0))
      self.dec2 = CBR(128, 64)
      self.up1 = nn.ConvTranspose3d(64, 32, 2, stride=2, output_padding=(1, 0, 0))
      self.dec1 = CBR(64, 32)
      self.out = nn.Conv3d(32, out_channels, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool1(e1))
        b = self.bottleneck(self.pool2(e2))

        # Upsample the bottleneck features
        upsampled_b = self.up2(b)

     # Determine the target spatial shape by taking the minimum size
        # along each spatial dimension (D, H, W)
        target_depth = min(upsampled_b.shape[2], e2.shape[2])
        target_height = min(upsampled_b.shape[3], e2.shape[3])
        target_width = min(upsampled_b.shape[4], e2.shape[4])

        # Crop both tensors to the target spatial dimensions
        cropped_upsampled_b = upsampled_b[:, :, :target_depth, :target_height, :target_width]

        cropped_e2 = crop_to_match(e2, upsampled_b)

        # ### Add print statements here to check shapes BEFORE concatenation ###
        print("Shape of upsampled_b before concat:", upsampled_b.shape)
        print("Shape of cropped_e2 before concat:", cropped_e2.shape)
        # ######################################################################


        # Concatenate along the channel dimension (dim=1)
        d2 = self.dec2(torch.cat([cropped_upsampled_b, cropped_e2], dim=1))

        # Repeat the process for the next level
        upsampled_d2 = self.up1(d2)

        # Determine the target spatial shape from the upsampled tensor for this level
        target_depth_up1, target_height_up1, target_width_up1 = upsampled_d2.shape[2:]

        # Crop the corresponding encoder tensor (e1) to match the target dimensions

        cropped_e1 = crop_to_match(e1, upsampled_d2)
        cropped_upsampled_d2 = crop_to_match(upsampled_d2, e1)

        # ### Add print statements here to check shapes BEFORE concatenation ###
        print("Shape of upsampled_d2 before concat:", upsampled_d2.shape)
        print("Shape of cropped_e1 before concat:", cropped_e1.shape)
        # ######################################################################

        d1 = self.dec1(torch.cat([cropped_upsampled_d2, cropped_e1], dim=1))

        return torch.sigmoid(self.out(d1))

# Inferencia (keep the rest of the inference code as is)
model = UNet3D(in_channels=1, out_channels=1)  # Instanciar la red
model.eval()

imagenes_gris = [img.convert('L') for img in imagenes_tif]
array_volumen=np.stack([np.array(img) for img in imagenes_gris])
volume_tensor = torch.tensor(array_volumen, dtype=torch.float32).unsqueeze(0).unsqueeze(0) / 255.0

with torch.no_grad():
    pred_mask = model(volume_tensor)  # Output shape: [1, 1, D, H, W]
    pred_mask_bin = (pred_mask > 0.5).float()

print(pred_mask.shape)


Shape of upsampled_b before concat: torch.Size([1, 64, 5, 510, 494])
Shape of cropped_e2 before concat: torch.Size([1, 64, 4, 510, 494])
Shape of upsampled_d2 before concat: torch.Size([1, 32, 9, 1020, 988])
Shape of cropped_e1 before concat: torch.Size([1, 32, 8, 1020, 988])
torch.Size([1, 1, 8, 1020, 988])


In [ ]:
# Inferencia

model = UNet3D(in_channels=1, out_channels=1)  # Instanciar la red
model.eval()

# Vamos a pasar las imágenes a escalas de grises (se supone en general mejor para el modelo, si es relevante el color se puedes dejar los tres canales)
imagenes_gris = [img.convert('L') for img in imagenes_tif]
array_volumen=np.stack([np.array(img) for img in imagenes_gris])
volume_tensor = torch.tensor(array_volumen, dtype=torch.float32).unsqueeze(0).unsqueeze(0) / 255.0

with torch.no_grad():
    pred_mask = model(volume_tensor)  # Output shape: [1, 1, D, H, W]
    pred_mask_bin = (pred_mask > 0.5).float()


Shape of upsampled_b before concat: torch.Size([1, 64, 5, 510, 494])
Shape of cropped_e2 before concat: torch.Size([1, 64, 4, 510, 494])
Shape of upsampled_d2 before concat: torch.Size([1, 32, 9, 1020, 988])
Shape of cropped_e1 before concat: torch.Size([1, 32, 8, 1020, 988])


In [ ]:
pred_mask

In [ ]:
z_spacing_mm = 25.4/96

voxel_volume_mm3 = (1 / 96)**2 * z_spacing_mm
volume_mm3 = torch.sum(pred_mask_bin).item() * voxel_volume_mm3
print(f"Volumen estimado: {volume_mm3:.2f} mm³")

Volumen estimado: 0.00 mm³


El entrenamiento de una red de este tipo requiere que los datos de entrenamiento estén etiquetados pixel a pixel, es decir, que se le indique cada pixel correspondiente a un defecto. Va a estar complicado.